# Iowa Liquor Sales Analysis Stage 1 Data Cleaning

# Setup and Imports 

In [1]:
# Common imports
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm import tqdm
import os

# Plotting imports and settings
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use("ggplot")
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (13,8)

# Metrics and useful imports
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split, cross_validate, StratifiedKFold

In [2]:
np.random.seed(0)

# THE SOURCE
Data on Iowa's alcohol sales from January 1, 2012, through April 1, 2023
These monthly updated Iowa liquor sale databases were obtained from the Alcoholic Beverages Division of the Iowa Department of Commerce.

This dataset includes information on the spirits purchased by Iowa Class "E" liquor licence holders between January 1, 2012, and the present, broken down by product and date of purchase. The dataset can be used to examine specific product sales of spirits at the retail level in Iowa as a whole.Grocery, liquor, convenience, etc. stores with a Class E liquor licence are able to sell alcohol in its original, unopened packaging for consumption off-site. 

In [3]:
IowaSales_raw = pd.read_csv("Iowa_Liquor_Sales2.csv", low_memory=False)
IowaSales_raw.shape

(26160915, 24)

In [4]:
size = IowaSales_raw.shape[0]
if(pd.options.display.max_info_rows <= size):
  pd.options.display.max_info_rows = size + 1

## Checking and removing null/missing values 

In [5]:
IowaSales_raw.isna().sum()

Invoice/Item Number            0
Date                           0
Store Number                   0
Store Name                     0
Address                    82627
City                       82626
Zip Code                   82693
Store Location           2495742
County Number            2028072
County                    159429
Category                   16974
Category Name              25040
Vendor Number                  9
Vendor Name                    7
Item Number                    0
Item Description               0
Pack                           0
Bottle Volume (ml)             0
State Bottle Cost             10
State Bottle Retail           10
Bottles Sold                   0
Sale (Dollars)                10
Volume Sold (Liters)           0
Volume Sold (Gallons)          0
dtype: int64

### Attribute Information
0. **`Invoice/Item Number`**    
1. **`Date`**                  
2. **`Store Number`**           
3. **`Store Name`**:            
4. **`Address`**: missing 82627 values,               
5. **`City`**: missing 82626 values,                    
6. **`Zip Code`**: missing 82693 values,                
7. **`Store Location`**: missing 2495742 values, split to longtidute and latitude         
8. **`County Number`**: missing 2028072 values
9. **`County`**: missing 159429 values, 
10. **`Category`**: missing 16974 values
11. **`Category Name`**: missing 25040 values
12. **`Vendor Number`**: missing 9 values
13. **`Vendor Name`**: missing 7 values, 
14. **`Item Number`**           
15. **`Item Description`**:       
16. **`Pack`**                
17. **`Bottle Volume (ml)`**     
18. **`State Bottle Cost`**: missing 10 values
19. **`State Bottle Retail`**: missing 10 values    
20. **`Bottles Sold`**            
21. **`Sale (Dollars)`**: missing 10 values        
22. **`Volume Sold (Liters)`**   
23. **`Volume Sold (Gallons)`** 

In [6]:
IowaSales_Copy_raw = IowaSales_raw.copy()
IowaSales_Copy_NoSaleNa = IowaSales_Copy_raw.dropna(subset=['Sale (Dollars)'])
IowaSales_Copy_NoSaleNa.isna().sum()



Invoice/Item Number            0
Date                           0
Store Number                   0
Store Name                     0
Address                    82627
City                       82626
Zip Code                   82693
Store Location           2495741
County Number            2028072
County                    159429
Category                   16974
Category Name              25040
Vendor Number                  9
Vendor Name                    7
Item Number                    0
Item Description               0
Pack                           0
Bottle Volume (ml)             0
State Bottle Cost              0
State Bottle Retail            0
Bottles Sold                   0
Sale (Dollars)                 0
Volume Sold (Liters)           0
Volume Sold (Gallons)          0
dtype: int64

In [7]:
# Try to locate some mapping for filling in some N/G data
store_locs_withna = IowaSales_Copy_raw[['Store Number', 'Store Location', 'Zip Code', 'City', 'County Number']]
store_locs = store_locs_withna.dropna()
store_locs_unique_idx =  store_locs['Store Number'].duplicated()
store_locs_unique_idx = (store_locs_unique_idx == False)
store_locs_unique = store_locs[store_locs_unique_idx]
store_locs_unique.head()

,Store Number,Store Location,Zip Code,City,County Number
0,2569,POINT (-91.630677 42.027805),52402,CEDAR RAPIDS,57.0
1,4167,POINT (-90.668138 42.504959),52001,DUBUQUE,31.0
2,4147,POINT (-92.474689 42.539076),50613,CEDAR FALLS,7.0
3,2556,POINT (-94.829962 43.402096),51334,ESTHERVILLE,32.0
5,3995,POINT (-92.430437 42.512768),50613,CEDAR FALLS,7.0


We are looking for a mapping to fill in the 'Store Location', 'Zip Code', 'City', and 'County Number' columns of the 'IowaSales_Copy_raw' DataFrame, which include missing data (N/G values). In order to accomplish this, a new DataFrame called "store_locs_withna" is created, including the pertinent columns, and any rows with missing values are dropped. Additionally, an exclusive index based on the "Store Number" column is created. The first five rows of the resulting DataFrame, which contain the unique locations of each store without any missing values, are then displayed. These unique rows are chosen based on this index. Using the 'Store Number' column as a key, this data can be utilised to complete the blanks in the original DataFrame.

In [8]:
store_locs_unique_ts = store_locs_unique.set_index('Store Number')
store_locs_unique_ts

,Store Location,Zip Code,City,County Number
Store Number,,,,
2569,POINT (-91.630677 42.027805),52402,CEDAR RAPIDS,57.0
4167,POINT (-90.668138 42.504959),52001,DUBUQUE,31.0
4147,POINT (-92.474689 42.539076),50613,CEDAR FALLS,7.0
2556,POINT (-94.829962 43.402096),51334,ESTHERVILLE,32.0
3995,POINT (-92.430437 42.512768),50613,CEDAR FALLS,7.0
...,...,...,...,...
9934,POINT (-90.455001 41.571059),52722,BETTENDORF,82.0
5372,POINT (-95.477367 43.184821),51346,HARTLEY,71.0
5370,POINT (-91.666012 42.745695),50606,ARLINGTON,33.0


In [9]:
# Try to map using existing mapping in data
IowaSales_Copy_raw.loc[IowaSales_Copy_raw['Store Location'].isna(),'Store Location'] = IowaSales_Copy_raw['Store Number'].map(lambda x: store_locs_unique_ts.at[x, 'Store Location'] if x in store_locs_unique_ts.index else np.NaN)
IowaSales_Copy_raw.loc[IowaSales_Copy_raw['Zip Code'].isna(),'Zip Code'] = IowaSales_Copy_raw['Store Number'].map(lambda x: store_locs_unique_ts.at[x, 'Zip Code'] if x in store_locs_unique_ts.index else np.NaN)
IowaSales_Copy_raw.loc[IowaSales_Copy_raw['City'].isna(),'City'] = IowaSales_Copy_raw['Store Number'].map(lambda x: store_locs_unique_ts.at[x, 'City'] if x in store_locs_unique_ts.index else np.NaN)
IowaSales_Copy_raw.loc[IowaSales_Copy_raw['County Number'].isna(),'County Number'] = IowaSales_Copy_raw['Store Number'].map(lambda x: store_locs_unique_ts.at[x, 'County Number'] if x in store_locs_unique_ts.index else np.NaN)
IowaSales_Copy_raw = IowaSales_Copy_raw.dropna(subset=['Sale (Dollars)'])

Using existing mappings from the "store_locs_unique_ts" DataFrame, we are attempting to fill in missing values in the "Store Location," "Zip Code," "City," and "County Number" columns of the "IowaSales_Copy_raw" DataFrame. This is accomplished by applying a lambda function to each missing value in the relevant columns, mapping each "Store Number" to, if available, its corresponding value in the "store_locs_unique_ts" DataFrame, or else assigning a missing value. To make sure the data is clear and suitable for analysis, the algorithm removes any rows with missing data in the "Sale (Dollars)" column.



In [10]:
IowaSales_Copy_raw.head()

,Invoice/Item Number,Date,Store Number,Store Name,Address,City,Zip Code,Store Location,County Number,County,...,Item Number,Item Description,Pack,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons)
0,INV-07993400005,10/16/2017,2569,HY-VEE FOOD STORE #2 / CEDAR RAPIDS,279 COLLINS ROAD NE,CEDAR RAPIDS,52402,POINT (-91.630677 42.027805),57.0,LINN,...,8828,LAUDER'S,6,1750,10.85,16.28,1,16.77,1.75,0.46
1,INV-07865000149,10/10/2017,4167,"IOWA STREET MARKET, INC.",1256 IOWA ST,DUBUQUE,52001,POINT (-90.668138 42.504959),31.0,DUBUQUE,...,44557,CRUZAN STRAWBERRY RUM,12,750,6.83,10.25,1,10.25,0.75,0.20
2,INV-07966000007,10/13/2017,4147,FAREWAY STORES #974 / CEDAR FALLS,214 N MAGNOLIA DR,CEDAR FALLS,50613,POINT (-92.474689 42.539076),7.0,BLACK HAWK,...,10791,CROWN ROYAL VANILLA,12,750,15.59,23.39,12,280.68,9.00,2.38
3,INV-07891600038,10/11/2017,2556,HY-VEE WINE AND SPIRITS / ESTHERVILLE,1033 CENTRAL AVENUE,ESTHERVILLE,51334,POINT (-94.829962 43.402096),32.0,EMMET,...,43028,ADMIRAL NELSON SPICED RUM,6,1750,11.55,17.33,6,103.98,10.50,2.77
4,INV-07976400007,10/13/2017,3629,WAL-MART 1965 / COUNCIL BLUFFS,3200 MANAWA DR,COUNCIL BLUFFS,51503,POINT (-95.848696 41.226557),78.0,POTTAWATTA,...,43028,ADMIRAL NELSON SPICED RUM,6,1750,11.55,17.33,12,207.96,21.00,5.55


In [11]:
# CLean up to re-categorize the Categories
IowaSales_Copy_raw['Category Name'] = IowaSales_Copy_raw['Category Name'].apply(lambda x: x.upper() if isinstance(x, str) else None)
IowaSales_Copy_raw['Category Name']= IowaSales_Copy_raw['Category Name'].apply(
    lambda x: None if x is None else 
    'WHISKY' if 'WHISKIES' in x else
    'WHISKY' if 'WHISKY' in x else
    'VODKA' if 'VODKA' in x else
    'RUM' if 'RUM' in x else
    'BEER' if 'BEER' in x else    
    'TEQUILA' if 'TEQUILA' in x else    
    'COCKTAIL' if 'COCKTAIL' in x else    
    'BRANDIES' if 'BRANDIES' in x else    
    'SCHNAPPS' if 'SCHNAPPS' in x else    
    'SCOTCH' if 'SCOTCH' in x else    
    'CORDIALS & LIQUEURS' if 'CORDIAL' in x else    
    'CORDIALS & LIQUEURS' if 'LIQUEUR' in x else    
    'CORDIALS & LIQUEURS' if 'TRIPLE SEC' in x else 
    'CORDIALS & LIQUEURS' if 'AMERICAN ALCOHOL' in x else
    'AMARETTO' if 'AMARETTO' in x else
    'SPIRIT SPECIALTY' if 'DISTILLED SPIRIT' in x else
    'SPIRIT SPECIALTY' if 'SPIRIT' in x else
    'SPIRIT SPECIALTY' if 'SPECIALTY' in x else
    'SPIRIT SPECIALTY' if 'CREME' in x else
    'SPIRIT SPECIALTY' if 'BOTTLED IN BOND BOURBON' in x else  
    'SPIRIT SPECIALTY' if 'HOLIDAY VAP' in x else      
    'GINS' if 'GIN' in x else    
    'OTHERS' if 'DELISTED ITEMS' in x else
    x)

In the dataset "IowaSales_Copy_raw," we purge and reclassify the "Category Name" field. The code raises the case of every text value in the column before replacing some of the data with fresh ones in accordance with predetermined criteria. The new classifications include "WHISKY," "VODKA," "RUM," "BEER," "TEQUILA," "COCKTAIL," "BRANDIES," "SCHNAPPS," "SCOTCH," "CORDIALS & LIQUEURS," "AMARETTO," "SPIRIT SPECIALTY," "GINS," and "OTHERS." The procedure of cleaning and reclassifying will make it simpler to analyse and summarise sales data by category.

In [12]:
IowaSales_Copy_raw.isna().sum()

Invoice/Item Number            0
Date                           0
Store Number                   0
Store Name                     0
Address                    82627
City                        7882
Zip Code                    7904
Store Location           1568518
County Number             184176
County                    159429
Category                   16974
Category Name              25040
Vendor Number                  9
Vendor Name                    7
Item Number                    0
Item Description               0
Pack                           0
Bottle Volume (ml)             0
State Bottle Cost              0
State Bottle Retail            0
Bottles Sold                   0
Sale (Dollars)                 0
Volume Sold (Liters)           0
Volume Sold (Gallons)          0
dtype: int64

In [13]:
def remove_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df_filtered = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    return df_filtered

# Remove outliers for each numerical column
numerical_columns = ['Pack', 'Bottle Volume (ml)', 'State Bottle Cost', 'State Bottle Retail', 'Bottles Sold', 'Sale (Dollars)', 'Volume Sold (Liters)', 'Volume Sold (Gallons)']
for column in numerical_columns:
    IowaSales_Copy_raw2 = remove_outliers_iqr(IowaSales_Copy_raw, column)

In [14]:
IowaSales_copybak = IowaSales_Copy_raw2.copy()

# Extract the Longtitue/Latitude
IowaSales_Copy_raw["Longitude"] = IowaSales_Copy_raw['Store Location'].map(lambda x: float(x.replace("(", "").replace(")","").split(" ")[1]) if isinstance(x, str) else np.NaN)
IowaSales_Copy_raw["Latitude"] = IowaSales_Copy_raw['Store Location'].map(lambda x: float(x.replace("(", "").replace(")","").split(" ")[2]) if isinstance(x, str) else np.NaN)



In [15]:
# Drop redundant columnes 
IowaSales_Copy = IowaSales_Copy_raw.drop(['Store Name','Address','Vendor Name','Item Description', 'Invoice/Item Number', "Store Location", "County"], axis=1)

In [16]:
# Further dropping some rare NA items after being filled with other existig mapping
IowaSales_Copy = IowaSales_Copy_raw.dropna(subset=['Sale (Dollars)', 'City', 'Zip Code', 'Longitude'])
IowaSales_Copy_Filled = IowaSales_Copy.fillna(method='ffill')
IowaSales_Copy_Filled.isna().sum()

Invoice/Item Number      0
Date                     0
Store Number             0
Store Name               0
Address                  0
City                     0
Zip Code                 0
Store Location           0
County Number            0
County                   0
Category                 0
Category Name            0
Vendor Number            0
Vendor Name              0
Item Number              0
Item Description         0
Pack                     0
Bottle Volume (ml)       0
State Bottle Cost        0
State Bottle Retail      0
Bottles Sold             0
Sale (Dollars)           0
Volume Sold (Liters)     0
Volume Sold (Gallons)    0
Longitude                0
Latitude                 0
dtype: int64

The 'dropna' function in Python is used to first remove any rows with empty values in the 'Sale (Dollars),' 'City,' 'Zip Code,' and 'Longitude' columns. The last non-missing value is then copied forward using the "fillna" function with the "method" parameter set to "ffill," filling in any remaining missing values in the dataset. The 'isna' function is used to examine the dataset for any further missing values before returning the overall number of missing values for each column. Overall, the algorithm makes sure that the dataset is accurate for analysis and modelling by making sure there are no missing values.


In [17]:
# Inspect data format before converting
IowaSales_Copy_Filled["Item Number"][IowaSales_Copy_Filled["Item Number"].map(lambda x: x.isdigit()) == False]

23906466    x904631
24940259    x904631
24952354    x904631
Name: Item Number, dtype: object

In [18]:
import datetime

# Converting 
IowaSales_Copy_Filled['Date'] = IowaSales_Copy_Filled['Date'].map(lambda x: datetime.datetime.strptime(x, '%m/%d/%Y'))
IowaSales_Copy_Filled['Month'] = IowaSales_Copy_Filled['Date'].map(lambda x: x.month)
IowaSales_Copy_Filled["City"] = IowaSales_Copy_Filled["City"].apply(lambda x: x.upper())
IowaSales_Copy_Filled["Zip Code"] = IowaSales_Copy_Filled["Zip Code"].map(lambda x: '51529' if x == '712-2' else x)
IowaSales_Copy_Filled["Item Number"] = IowaSales_Copy_Filled["Item Number"].map(lambda x: '904631' if x == 'x904631' else x)


In [19]:

IowaSales_Copy_Filled["Zip Code"] = IowaSales_Copy_Filled["Zip Code"].astype(int)
IowaSales_Copy_Filled["Item Number"] = IowaSales_Copy_Filled["Item Number"].astype(np.int32)

IowaSales_Copy_Filled["Pack Volume (ml)"] = IowaSales_Copy_Filled["Pack"] * IowaSales_Copy_Filled["Bottle Volume (ml)"]

In [20]:
IowaSales_Copy_Filled.columns

Index(['Invoice/Item Number', 'Date', 'Store Number', 'Store Name', 'Address',
       'City', 'Zip Code', 'Store Location', 'County Number', 'County',
       'Category', 'Category Name', 'Vendor Number', 'Vendor Name',
       'Item Number', 'Item Description', 'Pack', 'Bottle Volume (ml)',
       'State Bottle Cost', 'State Bottle Retail', 'Bottles Sold',
       'Sale (Dollars)', 'Volume Sold (Liters)', 'Volume Sold (Gallons)',
       'Longitude', 'Latitude', 'Month', 'Pack Volume (ml)'],
      dtype='object')

In [21]:
# Sampling for visualization
IowaSales_Main = IowaSales_Copy_Filled.drop(["Date", "Category Name"], axis=1)
IowaSales_Main.info(), IowaSales_Main.shape



<class 'pandas.core.frame.DataFrame'>
Int64Index: 24592365 entries, 0 to 26160913
Data columns (total 26 columns):
 #   Column                 Non-Null Count     Dtype  
---  ------                 --------------     -----  
 0   Invoice/Item Number    24592365 non-null  object 
 1   Store Number           24592365 non-null  int64  
 2   Store Name             24592365 non-null  object 
 3   Address                24592365 non-null  object 
 4   City                   24592365 non-null  object 
 5   Zip Code               24592365 non-null  int32  
 6   Store Location         24592365 non-null  object 
 7   County Number          24592365 non-null  float64
 8   County                 24592365 non-null  object 
 9   Category               24592365 non-null  float64
 10  Vendor Number          24592365 non-null  float64
 11  Vendor Name            24592365 non-null  object 
 12  Item Number            24592365 non-null  int32  
 13  Item Description       24592365 non-null  object 
 14  

(None, (24592365, 26))

In [22]:
IowaSales_Main.to_csv("Cleaneddatastage1.csv")